In [238]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split

In [239]:
import torch

import torchvision
import torchvision.transforms as transforms

import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [240]:
METHOD = False
device = 'cpu'

In [241]:

icouk_data_file = '..\\Data\\icouk_sighting_data_with_params.csv'
icop_data_file = '..\\Data\\icop_ahmed_2020_sighting_data_with_params.csv'
alrefay_data_file = '..\\Data\\alrefay_2018_sighting_data_with_params.csv'
allawi_data_file = '..\\Data\\schaefer_odeh_allawi_2022_sighting_data_with_params.csv'

icouk_data = pd.read_csv(icouk_data_file)
icop_data = pd.read_csv(icop_data_file)
alrefay_data = pd.read_csv(alrefay_data_file)
allawi_data = pd.read_csv(allawi_data_file)

data = pd.concat([icouk_data,icop_data,alrefay_data,allawi_data])

print(f"Loaded {data.shape[0]} rows")

data = data.drop(["Index","q","W","q'","W'",'Visibility'], axis = 1)

if METHOD: # method and methods columns, will be changed
    data = data.drop('Seen', axis = 1) # replaced by method column
    ptype = [r"Not_seen", r"Seen_eye", r"Seen_binoculars", r"Seen_telescope", r"Seen_ccd"] # CHANGE THIS
else:
    data=data.drop(['Method','Methods'], axis = 1)
    # List of label options
    ptype = [r"Seen", r"Not_seen"]

variable_list =  data.columns.tolist()
variable_list.remove('Seen')


Loaded 3626 rows


In [242]:
y = np.array(data['Seen'])
X = np.array(data[variable_list])

y[y == 'Seen'] = int(1)
y[y == 'Not_seen'] = int(0)


X_train, X_val_and_test, Y_train, Y_val_and_test = train_test_split(X, y, test_size=0.3)
X_val, X_test, Y_val, Y_test = train_test_split(X_val_and_test, Y_val_and_test, test_size=0.5)

In [243]:
# Define the neural network model
class CustomNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(CustomNN, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.fc2 = nn.Linear(hidden_size, hidden_size)
        self.fc3 = nn.Linear(hidden_size, output_size)  # Output layer with 1 neuron for binary classification
    
    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = torch.sigmoid(self.fc3(x))  # Sigmoid activation for binary classification
        return x
    
input_size = 19  # Number of input variables
hidden_size = 32  # Number of neurons in the hidden layers
output_size = 1 # for binary output
model = CustomNN(input_size, hidden_size, output_size)

# Define binary cross-entropy loss and Adam optimizer
criterion = nn.BCELoss() #nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [244]:
X_train_tensor = torch.from_numpy(X_train).float()
Y_train_tensor = torch.from_numpy(Y_train.astype("float64")).float()

In [245]:
# Paramaters for a loop
num_epochs = 100
batch_size = 32

for epoch in range(num_epochs):
    for i in range(0, len(X_train), batch_size):
        inputs = X_train_tensor[i:i+batch_size]
        targets = Y_train_tensor[i:i+batch_size]

        targets = targets.unsqueeze(1) # changes shape i guess
        
        # Forward pass
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        
        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
    # Print training loss after each epoch
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

Epoch [1/100], Loss: 70.0000
Epoch [2/100], Loss: 70.0000
Epoch [3/100], Loss: 70.0000
Epoch [4/100], Loss: 70.0000
Epoch [5/100], Loss: 70.0000
Epoch [6/100], Loss: 70.0000
Epoch [7/100], Loss: 70.0000
Epoch [8/100], Loss: 70.0000
Epoch [9/100], Loss: 70.0000
Epoch [10/100], Loss: 70.0000
Epoch [11/100], Loss: 70.0000
Epoch [12/100], Loss: 70.0000
Epoch [13/100], Loss: 70.0000
Epoch [14/100], Loss: 70.0000
Epoch [15/100], Loss: 70.0000
Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "c:\ProgramData\miniconda3\Lib\site-packages\IPython\core\interactiveshell.py", line 3526, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\Neil Power\AppData\Local\Temp\ipykernel_21428\1525706962.py", line 19, in <module>
    optimizer.step()
  File "c:\ProgramData\miniconda3\Lib\site-packages\torch\optim\optimizer.py", line 373, in wrapper
  File "c:\ProgramData\miniconda3\Lib\site-packages\torch\optim\optimizer.py", line 76, in _use_grad
  File "c:\ProgramData\miniconda3\Lib\site-packages\torch\optim\adam.py", line 163, in step
    return loss
        ^^^^^^^
  File "c:\ProgramData\miniconda3\Lib\site-packages\torch\optim\adam.py", line 311, in adam
    beta2: float,
  File "c:\ProgramData\miniconda3\Lib\site-packages\torch\optim\adam.py", line 375, in _single_tensor_adam
    else:
       ^^^
KeyboardInterrupt

During handling of the above exception, another exception occurred:

Traceback (most recent call l

In [ ]:
X_test_tensor = torch.Tensor(X_test)
predictions = model(X_test_tensor)

In [ ]:
#model = Sequential([
#    Dense(32, activation='softmax', input_shape=(18,)),
#    Dense(32, activation='softmax'),
#    Dense(1, activation='sigmoid'), # refelects catagory
#])

#model.compile(optimizer='adam',
#              loss='binary_crossentropy',
#              metrics=['accuracy'])
#
#hist = model.fit(X_train, Y_train,
#          batch_size=32, epochs=100,
#          validation_data=(X_val, Y_val))

In [ ]:
#model.evaluate(X_test, Y_test)[1]